In [1]:
import pandas as pd
from environments.envs import make_env, envs_list
from solvers.solvers import solve, solve_methods
import os
from tqdm import tqdm
from time import time

print(envs_list)
print(solve_methods)
env = make_env(envs_list[6])

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2023-12-10
['Ex10', 'Ex20', 'Ex30', 'Ex31', 'Ex40', 'froz_l_e', 'froz_l', 'taxi', 'rooms_1', 'rooms_2', 'rooms_3', 'rooms_4']
['mdpTB_VI', 'mdpTB_VIGS', 'mdpTB_PI', 'mdpTB_PIM', 'marmoteVI', 'marmoteVIGS', 'marmotePI', 'marmotePIM', 'gurobi_pl', 'gurobi_pl_dual']


In [7]:
columns = {solver : [0.]*len(envs_list) for solver in solve_methods}
df = pd.DataFrame(columns, index=envs_list)
df

,mdpTB_VI,mdpTB_VIGS,mdpTB_PI,mdpTB_PIM,marmoteVI,marmoteVIGS,marmotePI,marmotePIM,gurobi_pl,gurobi_pl_dual
Ex10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
froz_l_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
froz_l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
taxi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rooms_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rooms_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df.loc[:, 'mdpTB_VI']

Ex10        0.0
Ex20        0.0
Ex30        0.0
Ex31        0.0
Ex40        0.0
froz_l_e    0.0
froz_l      0.0
taxi        0.0
rooms_1     0.0
rooms_2     0.0
rooms_3     0.0
rooms_4     0.0
Name: mdpTB_VI, dtype: float64